# Modeling Car Insurance Claim Outcomes
## Intro
Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [1]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# import dataset
df = pd.read_csv('../datasets/car_insurance.csv')

In [2]:
# examine dataset
print(df.describe())
print(df.isna().sum())

                  id           age        gender  credit_score  \
count   10000.000000  10000.000000  10000.000000   9018.000000   
mean   500521.906800      1.489500      0.499000      0.515813   
std    290030.768758      1.025278      0.500024      0.137688   
min       101.000000      0.000000      0.000000      0.053358   
25%    249638.500000      1.000000      0.000000      0.417191   
50%    501777.000000      1.000000      0.000000      0.525033   
75%    753974.500000      2.000000      1.000000      0.618312   
max    999976.000000      3.000000      1.000000      0.960819   

       vehicle_ownership       married      children   postal_code  \
count       10000.000000  10000.000000  10000.000000  10000.000000   
mean            0.697000      0.498200      0.688800  19864.548400   
std             0.459578      0.500022      0.463008  18915.613855   
min             0.000000      0.000000      0.000000  10238.000000   
25%             0.000000      0.000000      0.000000  1

In [3]:
# filling missing values
df.fillna({
    "credit_score": df["credit_score"].mean(),
    "annual_mileage": df["annual_mileage"].mean()
}, inplace=True)


In [4]:
# model preparation
models = []
features = df.drop(columns=['outcome', 'id']).columns

In [5]:
# import model
from statsmodels.formula.api import logit

# model per feature
models = []
for i in features:
    model = logit(f'outcome ~ {i}', data=df).fit()
    models.append(model)

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

In [6]:
# measuring performance
accuracies = []

# Loop through models
for feature in range(0, len(models)):
    # Compute the confusion matrix
    conf_matrix = models[feature].pred_table()
    # True negatives
    tn = conf_matrix[0,0]
    # True positives
    tp = conf_matrix[1,1]
    # False negatives
    fn = conf_matrix[1,0]
    # False positives
    fp = conf_matrix[0,1]
    # Compute accuracy
    acc = (tn + tp) / (tn + fn + fp + tp)
    accuracies.append(acc)
accuracies

[np.float64(0.7747),
 np.float64(0.6867),
 np.float64(0.7771),
 np.float64(0.6867),
 np.float64(0.7425),
 np.float64(0.7054),
 np.float64(0.7351),
 np.float64(0.6867),
 np.float64(0.6867),
 np.float64(0.6867),
 np.float64(0.6867),
 np.float64(0.6904),
 np.float64(0.6867),
 np.float64(0.6867),
 np.float64(0.6867),
 np.float64(0.6867)]

In [7]:
# Find the feature with the largest accuracy
best_feature = features[accuracies.index(max(accuracies))]

# Create best_feature_df
best_feature_df = pd.DataFrame({"best_feature": best_feature,
                                "best_accuracy": max(accuracies)},
                                index=[0])
best_feature_df

,best_feature,best_accuracy
0,driving_experience,0.7771
